## Things to add
 - multi shot prompt 
 - chain of thought prompt 
 - saving responses 
 - chat memory (maybe)

# Model Testing 

**Here are my PC specs :)** 
 - CPU: AMD 5900x (12 core, 24 thread)
 - GPU: RTX 3080ti (12gb VRAM)
 - MEMORY: 32gb 3600mhz
 - STORAGE: 2 x 2TB NVME

In [10]:
# import the stuff we need
import langchain
import ollama

## model selections 
 - Qwen 2.5 coder (0.5b) 
 - Qwen 2.5 coder (32b) 
 - Wizardcoder (33b)


In [23]:
# pick what model to use 
model1 = 'qwen2.5-coder:0.5b'
model2 = 'qwen2.5-coder:32b'
model3 = 'wizardcoder:33b'

## Prompt templates
 - one shot 
 - multi shot
 - chain of thought 
---

In [12]:
# import the prompt block
from langchain.prompts import PromptTemplate

In [13]:
oneShot = PromptTemplate(
    input_variables=["method_code"],
    template='''
Write a clear and professional method header in Javadoc style. I will provide some examples.

method_code: 
public int factorial(int n) {{
    if (n < 0) {{
        throw new IllegalArgumentException("Input must be non-negative.");
    }}
    int result = 1;
    for (int i = 2; i <= n; i++) {{
        result *= i;
    }}
    return result;
}}

good output:
/**
 * Calculates the factorial of a given non-negative integer.
 *
 * @param n the number to compute the factorial of
 * @return the factorial of n
 * @throws IllegalArgumentException if n is negative
 */

now given {method_code}:

write a Javadoc header for the method above.

'''
)


## Sample code 


In [21]:
removeFromBST = oneShot.format(method_code=
    """private Node doRemove(Node r, String s, Node before, boolean removeOnce, Integer val) {
		if(r == null) {
			lastRemoved = null;
			return null;
		}
		int c = s.compareTo(r.string);
		if(c < 0) r.left = doRemove(r.left, s, before, removeOnce, val);
		else if(c > 0) r.right = doRemove(r.right, s, r, removeOnce, val);
		else {
			if(val != null && val != r.count) return r;
			lastRemoved = r.count;
			if(removeOnce && r.count > 1) {
				r.count--;
				return r;
			}
			if(before.next != r) {
				before = r.left;
				while(before.right != null) before = before.right;
			}
			version++;
			numEntries--;
			if(r.left == null) {
				before.next = r.next;
				return r.right;
			}
			if(r.right == null) {
				before.next = r.next;
				return r.left;
			}
			
			Node successor = r.right;
			Node prev = r;
			while(successor.left != null) {
				prev = successor;
				successor = successor.left;
			}
			r.string = successor.string;
			r.count = successor.count;
			
			if(prev.left == successor) prev.left = successor.right;
			else prev.right = successor.right;
			
			r.next = successor.next;


		}
		return r;
"""
)

In [14]:
# add some content 
shortMethod = oneShot.format(
    method_code = """
    private void connect(HexPiece p) {
HexCoordinate h = p.getLocation();
for (HexDirection d : HexDirection.values()) {
HexCoordinate h2 = d.move(h);
HexPiece p2 = findPiece(h2);
if (p2 != null) {
p.neighbors[d.ordinal()] = p2;
p2.neighbors[d.reverse().ordinal()] = p;
}
}
}
                         """)

In [15]:
longMethodWithHelper = oneShot.format(method_code="""\
private void rehash() {{

    int newCapacity = Primes.nextPrime(table.length * 2);
    HexPiece[] oldTable = table;
    table = new HexPiece[newCapacity];
    size = 0;

    for (HexPiece head : oldTable) {{
        if (head == null) continue;
        HexPiece current = head;
        do {{
            HexPiece next = current.nextInChain;
            int newIndex = locate(current.location);

            if (table[newIndex] == null) {{
                current.nextInChain = current;
                table[newIndex] = current;
            }} else {{
                current.nextInChain = table[newIndex].nextInChain;
                table[newIndex].nextInChain = current;
            }}
            size++;
            current = next;
        }} while (current != head);
    }}
}}

public boolean add(HexPiece p) {{
    assert wellFormed() : "Invariant broken before add";

    if (p == null || p.location == null || p.terrain == null)
        throw new NullPointerException("Piece, location, and terrain must not be null");

    int index = locate(p.location);
    HexPiece head = table[index];

    if (head != null) {{
        HexPiece current = head;
        do {{
            if (current.location.equals(p.location)) {{
                current.terrain = p.terrain;
                version++;
                assert wellFormed() : "Invariant broken after add (update)";
                return false;
            }}
            current = current.nextInChain;
        }} while (current != head);

        p.nextInChain = head.nextInChain;
        head.nextInChain = p;
    }} else {{
        p.nextInChain = p;
        table[index] = p;
    }}

    connect(p);
    size++;
    version++;

    if (size >= table.length) {{
        rehash();
    }}

    assert wellFormed() : "Invariant broken after add (insert)";
    return true;
}}
""")


# Evaluating outputs

### Qwen2.5 coder (0.5b)

In [22]:
# Run model
from ollama import chat

response = chat(model=model1, messages=[{'role': 'user', 'content': removeFromBST}])
print(response['message']['content'])


/**
 * Removes a node from the list, returning the next node in its original position.
 *
 * @param n the node to remove from the list
 * @return the next node in its original position
 */
public Node doRemove(Node r, String s, Node before, boolean removeOnce) {
    if (r == null) return null;

    int c = s.compareTo(r.string);
    if (c < 0) r.left = doRemove(r.left, s, before, removeOnce, val);
    else if (c > 0) r.right = doRemove(r.right, s, r, removeOnce, val);
    else {
        if(val != null && val != r.count) return r;
        lastRemoved = r.count;
        if(removeOnce && r.count > 1) {
            r.count--;
            return r;
        }
        if(before.next != r) {
            before = r.left;
            while(before.right != null) before = before.right;
        }
        version++;
        numEntries--;
        if(r.left == null) {
            before.next = r.next;
            return r.right;
        }
        if(r.right == null) {
            before.next = r.next;

### Qwen 2.5 coder (32b)

In [19]:
# Run model
response = chat(model=model2, messages=[{'role': 'user', 'content': longMethodWithHelper}])
print(response['message']['content'])


Certainly! Below is the Javadoc comment for the `rehash` method based on the provided code snippet and examples:

```java
/**
 * Rehashes the table to double its capacity, redistributing existing elements.
 * This method recalculates the new capacity using the next prime number greater
 * than twice the current table length. It then re-inserts all elements into the
 * new table based on their updated hash indices.
 *
 * @throws AssertionError if the invariant of the data structure is broken during rehashing
 */
private void rehash() {
    int newCapacity = Primes.nextPrime(table.length * 2);
    HexPiece[] oldTable = table;
    table = new HexPiece[newCapacity];
    size = 0;

    for (HexPiece head : oldTable) {
        if (head == null) continue;
        HexPiece current = head;
        do {
            HexPiece next = current.nextInChain;
            int newIndex = locate(current.location);

            if (table[newIndex] == null) {
                current.nextInChain = current;
  

### Wizard Coder (15B)

In [24]:
# Run model
response = chat(model=model3, messages=[{'role': 'user', 'content': longMethodWithHelper}])
print(response['message']['content'])

/**
 * Rehashes the hash table when it gets too full. This involves creating a new, larger table and rehashing all existing elements into it.
 */
private void rehash() {
    int newCapacity = Primes.nextPrime(table.length * 2);
    HexPiece[] oldTable = table;
    table = new HexPiece[newCapacity];
    size = 0;
    
    for(HexPiece head : oldTable) {
        if (head == null) continue;
        HexPiece current = head;
        do {
            HexPiece next = current.nextInChain;
            int newIndex = locate(current.location);
            
            if(table[newIndex] == null) {
                current.nextInChain = current;
                table[newIndex] = current;
            } else {
                current.nextInChain = table[newIndex].nextInChain;
                table[newIndex].nextInChain = current;
            }
            size++;
            current = next;
        } while(current != head);
    }
}

/**
 * Adds a new HexPiece to the hash table. If a Piece with the sa